Partioning Genetic Algorithms
====================

By Scott O'Connor
---------------------

This is a quick look at partitioning GA's

In [1]:
import numpy as np
import operator
from copy import deepcopy
import matplotlib.pyplot as plt

<h1>Organism Class</h1>
Stores the following:
<ul>
<li>partition</li>
<li>fitness-Value assigned by the Envirnoment it is in</li>
<ul>


In [2]:
class Organism(object):
    '''
    An organism is defined by its partition and has resulting fitness.
    Fitness is determined by the enviroment it is in.
    '''
    def __init__(self, partition):
        self.partition = partition
        self.fitness = None

<h1>Population Class</h1>
Stores the following:
<ul>
<li>Population Size - to be used in indexing</li>
<li>Partition Length </li>
<li>Organisms - a list of Organism Objects</li>
<li>Minium Fitness - what is the smallest fitness</li>
<ul>


In [3]:
class Population(object):
    """
    Contains all the organism for one generations
    """
    def __init__(self, population, nprtl):
        self.population_size = len(population)
        self.nprtl = nprtl
        self.organisms = population 
        self.min_fitness = self.__Min_Fitness()
        
        
    def __Min_Fitness(self):
        """
        Calculates Min Fitness of that generation   
        """
        min_fitness = self.organisms[0].fitness    
        for ii in range(self.population_size):
            if (self.organisms[ii].fitness != None):
                if (self.organisms[ii].fitness < min_fitness):
                    min_fitness = self.organisms[ii].fitness
        return min_fitness

<h1>Envirnoment Class</h1>
Stores the following
<ul>
<li>Partition Width </li>
<li>Number of Bins  </li>
<li>Population Size</li>
<li>Weights</li>
<li>Generations</li>
<li>Number of Generations</li>
</ul>

Function: inital Population - create a population of organisms

Function: Create_New_Generation - Creates a new generation of organisms

In [24]:
class Environment(object):
    def __init__(self,weights,nprtl,pop_size,nbins):
        self.nprtl = nprtl
        self.nbins = nbins
        self.population_size = pop_size
        self.weights = weights
        self.generation = self.__initial_population()
        self.num_generations = 0
        
    def __initial_population(self):
        '''Creates an initial population of organisms'''
        population = []
        for ii in range(self.population_size):
            partition = np.random.randint(self.nbins, size = self.nprtl)            
            organism = Organism(partition)
            organism.fitness = Fitness_Multi_Bin(organism,self.nbins,self.weights)
            population.append(organism)
        generation = Population(population,self.nprtl)
        return [generation]
              
    def Create_New_Generation(self):
        '''
        Create New Generation by performing the following steps
                1) orders the population from min fitness to max fitness
                2) Selects parents from ordered list to 'Mate'
                3) assigns fitness to new organism and repeat 
        '''
        self.num_generations +=1
        g_idx = self.num_generations - 1
        current_gen = deepcopy(self.generation[g_idx])
        regen = order_generation(current_gen)
        new_pop = []
        for ii in range(int(self.generation[g_idx].population_size/2)):
            #Select Parents
            #p_sel = Roulette_Wheel_Mate_Selection(self.generation[g_idx].population_size)           
            p_sel = Boltzman_Tournament_Selection(regen)
            #Mate Parents            
            new_org1, new_org2 = Mating_Single_Cross_Over(regen.organisms[p_sel[0]],regen.organisms[p_sel[1]])
            
            #Mutation
            Mutation_Organism(new_org1,self.nprtl,self.nbins)
            Mutation_Organism(new_org2,self.nprtl,self.nbins)
            
            #Set Children's fitness            
            new_org1.fitness = Fitness_Multi_Bin(new_org1,self.nbins,self.weights)
            new_org2.fitness = Fitness_Multi_Bin(new_org2,self.nbins,self.weights)
            
            #Add to new Generations population            
            new_pop.append(new_org1)
            new_pop.append(new_org2)
            
        #Create new Population object and append it to the generation class Variable
        new_gen = Population(new_pop,self.nprtl)
        self.generation.append(new_gen)

<H1>Fitness Function</H1>
Calculates absolutue value of error of perfect partitioning
Ideal Soluation is 
<br>
$\sum| F_i - F_{ideal}| $

In [25]:
def Fitness_Multi_Bin(organism,NBINS,weights):
    bins = np.zeros(NBINS)
    for ii in range(len(organism.partition)):
        bin_num = organism.partition[ii]        
        bins[bin_num] =bins[bin_num] + weights[ii]
    idealsol = sum(weights)/NBINS
    return sum(abs(bins-idealsol))

<H1>Ordering</h1>
Each generation is ordered from smallest fitnes to largest


In [26]:
def order_generation(current_gen):
    '''Orders Population from min to max fitness'''
    current_gen.organisms.sort(key = lambda x: x.fitness)
    return current_gen

<H1>Roulette Wheel Mate Selection</H1>
Probability of selection for mating is based on position of chromosome ranked by fitness

$p(chromosome) = \frac{N_{prt}-(R_{rank}-1)}{\sum positions}$ 

Example with four chromosomes<br>
$p(chromosome 1) = \frac{4}{1+2+3+4} = 0.4$<br> 
$p(chromosome 2) = \frac{3}{1+2+3+4} = 0.3$<br>
$p(chromosome 3) = \frac{2}{1+2+3+4} = 0.2$<br>
$p(chromosome 4) = \frac{1}{1+2+3+4} = 0.1$<br>

<h5>Source: Genetic Algorithms in Electromagnetics, ch2 ~ Randy L. Haupt and Douglas H Werner</h5>

In [27]:
def Roulette_Wheel_Mate_Selection(pop_size):
    parents = np.arange(pop_size)+1
    prob = np.zeros(pop_size)
    for ii in range(pop_size):
        prob[ii] = parents[ii]/np.sum(parents)
    prob[::-1] = prob #niffty way to reverse array
    probsum = np.cumsum(prob)
    rand1 = np.random.rand(1)
    rand2 = np.random.rand(1)
    parent1 = np.digitize(rand1,probsum)
    parent2 = np.digitize(rand2,probsum)
    if parent1 == parent2:
        parent2 = parent2 + 1
    return [parent1,parent2]

In [28]:
Roulette_Wheel_Mate_Selection(4)

[array([4], dtype=int64), array([5], dtype=int64)]

<H1>Bolzman Weighted Selection</H1>
Selection has a temperature factor associated with it

$p(chromosom)=\frac{\exp^{-B*F_i/T}}{\sum \exp^{-B*F_i/T}}$

<h5>Source: ftp://publications.ai.mit.edu/ai-publications/pdf/AIM-1345.pdf</h5>

In [29]:
def Boltzmann_Tournament_Selection(pop):
    B = 1
    T = 100000    
    prob = np.zeros(pop.population_size)
    boltzman = np.zeros(pop.population_size)
    for ii in range(pop.population_size):
        boltzman[ii] = np.exp(-B*pop.organisms[ii].fitness/T)
    
    #find probability
    for ii in range(pop.population_size):
        prob[ii] = boltzman[ii]/sum(boltzman)
    probsum = np.cumsum(prob)
    rand1 = np.random.rand(1)
    rand2 = np.random.rand(1)
    parent1 = np.digitize(rand1,probsum)
    parent2 = np.digitize(rand2,probsum)
    if parent1 == parent2:
        parent2 = parent2 + 1
    return [parent1,parent2]

<h1>Mating Single Cross Over</h1>

In [30]:
def Mating_Single_Cross_Over(parent1,parent2):
    '''
    Generates offspring using a single cross over point.
    Make Selection comes from a roulette wheel approach
    '''
    # Generate Cross Over Point
    cross_over_point = np.random.randint(len(parent1.partition))
    
    #Deep Copy Parents
    partition1 = deepcopy(parent1.partition)
    partition2 = deepcopy(parent2.partition)
    
    #Deep Copy Opposite parent 'Chunk' up to cross over point
    partition1[0:cross_over_point] = deepcopy(parent2.partition[0:cross_over_point])
    partition2[0:cross_over_point] = deepcopy(parent1.partition[0:cross_over_point])
    
    #Create Two new Child Organisms and return them 
    organism1 = Organism(partition1)
    organism2 = Organism(partition2)
    return organism1, organism2

In [31]:
#Test Case:
parent1 = Organism([0,1,2,3,4,5,6,7])
parent2 = Organism([10,11,12,13,14,15,16,17])
offspring1,offspring2 = Mating_Single_Cross_Over(parent1,parent2)

<H1>Mutations</H1>
By randomly mutating a single chromosome the GA was futher improved

In [32]:
def Mutation_Organism(organism,nprtl,nbins):
    '''
    Randomly will mutate a gene from a parents offspring before their fitness is tested
    '''
    if (organism.fitness == None)&(np.random.rand(1) > 0.95):
        rand_index = int(np.random.rand(1)*nprtl)
        organism.partition[rand_index] = np.random.randint(nbins)

<H1>Weights</H1>
Partitioning Sets with Genetic Algorithms
http://helpdesk.cs.uno.edu/people/faculty/bill/Partitioning-Sets-FLAIRS-2000.pdf

In [33]:
weights = [ 3380, 1824, 1481, 2060, 1225, 836, 1363, 2705, 4635, 648, 2588, 3380, 1952, 3832, 3176, 2316, 2479, 3433, 3519, 1363, 1824, 3305, 2156, 3305, 3049, 3980, 2787, 4635, 4068, 2992, 5932, 528, 3304, 4107]

In [34]:
#Program Parameters
NPRTL = 34              #Partition Size - There are 34 numbers in the weight array
NBINS = 10              #Number of Bins
POPULATION_SIZE = 200   #Population Size
NGEN = 100               #Number of Generations

In [35]:
#Create an object GA1 that has an initalized population
GA1 = Environment(weights,NPRTL,POPULATION_SIZE,NBINS)

In [36]:
GA1.generation[0].min_fitness

17007.0

In [37]:
#print('Before Sort')
#for ii in range(GA1.generation[0].population_size):
#    print(GA1.generation[0].organisms[ii].fitness)
#    print(GA1.generation[0].organisms[ii].partition)
#order_generation(GA1.generation[0])
#print('After Sort')
#or ii in range(GA1.generation[0].population_size):
#    print(GA1.generation[0].organisms[ii].fitness)
#    print(GA1.generation[0].organisms[ii].partition)

In [23]:
#Loop to run the program 
fitplot = np.zeros(NGEN)
for ii in range(NGEN):
    GA1.Create_New_Generation()
    fitplot[ii] = GA1.generation[ii].min_fitness
    print(ii,GA1.generation[ii].min_fitness)

IndexError: list index out of range

In [19]:
#plot the results
plt.plot(fitplot)
plt.ylabel('Fitness')
plt.xlabel('Generations')
plt.show()